In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv('/Users/parthsdambhare/Documents/Enc/Spam_sms_detection/spam.csv' , encoding='ISO-8859-1')
print(data.columns)
print(data.head())


Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')
     v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN  


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    data['v2'],  # Assuming the column for email content is named 'text'
    data['v1'],  # Assuming the column for labels is named 'label'
    test_size=0.2,
    random_state=42
)

In [5]:
# Prepare the data for FastText
train_data = X_train.to_frame().assign(v1=y_train).rename(columns={'v2': 'v2'})
train_data['v1'] = train_data['v1'].apply(lambda x: f'__label__{x}')  # Prefix labels

test_data = X_test.to_frame().assign(v1=y_test).rename(columns={'v2': 'v2'})
test_data['v1'] = test_data['v1'].apply(lambda x: f'__label__{x}')  # Prefix labels

# Save to text files for FastText
train_data[['v1', 'v2']].to_csv('train.txt', sep='\t', index=False, header=False)
test_data[['v1', 'v2']].to_csv('test.txt', sep='\t', index=False, header=False)



In [17]:
import fasttext

# Train the FastText model
model = fasttext.train_supervised('train.txt', lr=5e-3, epoch=3000, wordNgrams=2)


Read 0M words
Number of words:  13635
Number of labels: 2
Progress: 100.0% words/sec/thread: 1077233 lr:  0.000000 avg.loss:  0.007055 ETA:   0h 0m 0s


In [18]:
# Evaluate the model
nexamples, precision, recall = model.test('test.txt')

print(f"Number of examples: {nexamples}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


Number of examples: 1115
Precision: 0.9821
Recall: 0.9821


In [19]:
# Example of predicting spam or ham
sample_text = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.   "
prediction = model.predict(sample_text)

print(f"Predicted label: {prediction[0][0]}")


Predicted label: __label__spam
